In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
from numpy import savetxt

In [3]:
# Load data from emnist
data = sio.loadmat("emnist-byclass.mat")['dataset']

In [4]:
# data[0][0][0] - 3 options (train, test, mapping)
# data[0][0][0][0][0][0] - 3 options (images, labels, writers)
# data[0][0][0][0][0][0][0]

for i in range(0):
    label = data[0][0][0][0][0][1][1000 + i]

    pixels = data[0][0][0][0][0][0][1000 + i].reshape((28, 28))

    plt.title('Label is {label}'.format(label=label))
    plt.imshow(pixels, cmap='gray')
    plt.show()

In [5]:
from sklearn.model_selection import train_test_split

trainSize = 0.1

X_train_data = data[0][0][0][0][0][0]
y_train_data = data[0][0][0][0][0][1]

X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train_data, train_size=trainSize, random_state=0)


In [6]:
X_test = data[0][0][1][0][0][0]
y_test = data[0][0][1][0][0][1]

In [7]:
x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, input_dim=784, activation='linear'))
model.add(tf.keras.layers.Dense(62, activation='softmax'))

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
2182/2182 [==============================] - 3s 1ms/step - loss: 39.2800 - accuracy: 0.5365
Epoch 2/3
2182/2182 [==============================] - 2s 1ms/step - loss: 12.1094 - accuracy: 0.5649
Epoch 3/3
2182/2182 [==============================] - 2s 1ms/step - loss: 4.6501 - accuracy: 0.5419


In [11]:
model.save('symbolRecon_Lin.model')

INFO:tensorflow:Assets written to: symbolRecon_Lin.model\assets


In [12]:
model = tf.keras.models.load_model('symbolRecon_Lin.model')

In [13]:
loss, accurary = model.evaluate(X_test, y_test)

3636/3636 [==============================] - 3s 792us/step - loss: 3.4578 - accuracy: 0.5310


In [14]:
savetxt('ytest.csv', y_test, delimiter=',')

In [15]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
print(y_pred.shape)

3636/3636 [==============================] - 3s 688us/step
(116323,)


In [16]:
savetxt('ypred_Lin.csv', y_pred, delimiter=',')
print(y_pred.shape)

(116323,)


In [17]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

correct = 0
for i in range(len(cm)):
    correct += cm[i][i]

r = round((correct) / np.sum(cm), 4)
print(f'Classifiation Rate: {r}')

savetxt('data_Lin.csv', cm, delimiter=',')

Classifiation Rate: 0.531
